In [2]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertForQuestionAnswering
import gc

In [3]:
# tokeniser = BertTokenizer.from_pretrained(model_dir)  # Add specific options if needed
model = BertForQuestionAnswering.from_pretrained('bert-base-chinese')
tokeniser = BertTokenizer.from_pretrained('bert-base-chinese')

model.eval()

if torch.cuda.is_available():

    dirve ='cuda'
else:
    dirve ='cpu'
print('use ',dirve)
model.to(dirve)
print('加载模型完成')

100%|██████████| 109540/109540 [00:00<00:00, 4207517.31B/s]

use  cpu
加载模型完成


In [13]:
#释放内存
# del model
# gc.collect()

44

In [1]:
# model

In [4]:
q='明天天气怎么样'

context="""

明天天气不错 我们不如一起出去吧
"""
 


question='[CLS] '+ q + ' [SEP]'

context= context+ ' [SEP]'

len_line_1 = len(tokeniser.tokenize(question))
len_line_2 = len(tokeniser.tokenize(context))


input_mask =question + context

# text = question + context

#         print('text',text)
tokenized_text = tokeniser.tokenize(input_mask)
print('tokenized_text',tokenized_text)

indexed_tokens = tokeniser.convert_tokens_to_ids(tokenized_text)
# input_mask_tokens = tokeniser.convert_tokens_to_ids(input_mask)
segments_ids = ([0] * len_line_1) + ([1] * len_line_2)
input_mask_tokens = [1]*len(indexed_tokens)

print(torch.long)
print(indexed_tokens)

input_mask_tensors= torch.tensor([input_mask_tokens], dtype=torch.long).to(dirve)
tokens_tensor = torch.tensor([indexed_tokens], dtype=torch.long).to(dirve)
segments_tensors = torch.tensor([segments_ids], dtype=torch.long).to(dirve)

print('tokens_tensor',tokens_tensor)
print('segments_tensors',segments_tensors)
print('input_mask_tensors',input_mask_tensors)

#         print('tokens_tensor',tokens_tensor)
#         print('segments_tensors',segments_tensors)
#这里使用模型进行预测
batch_start_logits, batch_end_logits = model(tokens_tensor, segments_tensors,input_mask_tensors)
print('batch_start_logits',batch_start_logits)
print('batch_end_logits',batch_end_logits)

tokenized_text ['[CLS]', '明', '天', '天', '气', '怎', '么', '样', '[SEP]', '明', '天', '天', '气', '不', '错', '我', '们', '不', '如', '一', '起', '出', '去', '吧', '[SEP]']
torch.int64
[101, 3209, 1921, 1921, 3698, 2582, 720, 3416, 102, 3209, 1921, 1921, 3698, 679, 7231, 2769, 812, 679, 1963, 671, 6629, 1139, 1343, 1416, 102]
tokens_tensor tensor([[ 101, 3209, 1921, 1921, 3698, 2582,  720, 3416,  102, 3209, 1921, 1921,
         3698,  679, 7231, 2769,  812,  679, 1963,  671, 6629, 1139, 1343, 1416,
          102]])
segments_tensors tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])
input_mask_tensors tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])
batch_start_logits tensor([[ 0.1218,  0.4153,  0.2926, -0.3432, -0.2488,  0.1757,  0.2033,  0.4634,
          0.4001,  0.4965,  0.1890, -0.4528, -0.1047,  0.8451,  0.4144,  0.6293,
          0.7450,  0.1684,  0.1145,  0.1997,  0.3853,  0.2564,  0.3872,  0.4191,
          0

In [5]:
def get_best_indexes(logits, n_best_size):
  """Get the n-best logits from a list."""
  index_and_score = sorted(enumerate(logits), key=lambda x: x[1], reverse=True)
  print('index_and_score',index_and_score)

  best_indexes = []
  for i in range(len(index_and_score)):
    if i >= n_best_size:
      break
    best_indexes.append(index_and_score[i][0])
  return best_indexes

In [6]:
n_best_size = 20

In [7]:
start_logits = batch_start_logits[0].detach().cpu().tolist()
start_logits

[0.12179139256477356,
 0.4152701497077942,
 0.29259419441223145,
 -0.34319061040878296,
 -0.24883460998535156,
 0.17570917308330536,
 0.20331519842147827,
 0.46344947814941406,
 0.400058388710022,
 0.49645471572875977,
 0.1890077292919159,
 -0.4527963399887085,
 -0.10467958450317383,
 0.8450657725334167,
 0.4144056439399719,
 0.629267156124115,
 0.7450343370437622,
 0.16835471987724304,
 0.11452361941337585,
 0.19966839253902435,
 0.3853258490562439,
 0.2563633918762207,
 0.3872082531452179,
 0.4190806746482849,
 0.40005743503570557]

In [8]:
end_logits = batch_end_logits[0].detach().cpu().tolist()
end_logits

[0.2933330535888672,
 -0.3933124542236328,
 0.11820844560861588,
 0.2861650586128235,
 0.6896135210990906,
 0.2185426950454712,
 -0.22814348340034485,
 0.383070170879364,
 0.5493966341018677,
 0.6373893618583679,
 0.43379515409469604,
 0.4106791019439697,
 0.9121344685554504,
 0.5905374884605408,
 0.5550487637519836,
 0.5825162529945374,
 0.1551828533411026,
 0.6593695878982544,
 0.36921340227127075,
 0.14490613341331482,
 0.3039102554321289,
 0.6883697509765625,
 0.517236590385437,
 0.609614372253418,
 0.5493967533111572]

In [9]:
start_indexes = get_best_indexes( start_logits, n_best_size)
end_indexes = get_best_indexes(end_logits, n_best_size)


print('start_indexes',start_indexes)
print('end_indexes',end_indexes)

index_and_score [(13, 0.8450657725334167), (16, 0.7450343370437622), (15, 0.629267156124115), (9, 0.49645471572875977), (7, 0.46344947814941406), (23, 0.4190806746482849), (1, 0.4152701497077942), (14, 0.4144056439399719), (8, 0.400058388710022), (24, 0.40005743503570557), (22, 0.3872082531452179), (20, 0.3853258490562439), (2, 0.29259419441223145), (21, 0.2563633918762207), (6, 0.20331519842147827), (19, 0.19966839253902435), (10, 0.1890077292919159), (5, 0.17570917308330536), (17, 0.16835471987724304), (0, 0.12179139256477356), (18, 0.11452361941337585), (12, -0.10467958450317383), (4, -0.24883460998535156), (3, -0.34319061040878296), (11, -0.4527963399887085)]
index_and_score [(12, 0.9121344685554504), (4, 0.6896135210990906), (21, 0.6883697509765625), (17, 0.6593695878982544), (9, 0.6373893618583679), (23, 0.609614372253418), (13, 0.5905374884605408), (15, 0.5825162529945374), (14, 0.5550487637519836), (24, 0.5493967533111572), (8, 0.5493966341018677), (22, 0.517236590385437), (10,

In [10]:
print('tokenized_text',tokenized_text)

text = tokenized_text[1:3]
print('预测结果',text)
max_answer_length=20
for start_index in start_indexes:
    for end_index in end_indexes:
        # We could hypothetically create invalid predictions, e.g., predict
        # that the start of the span is in the question. We throw out all
        # invalid predictions.
                    
#         print('*'*20)
#         print('start_index',start_index)
#         print('end_index',end_index)
        start= int(start_index)
        end =int(end_index)
        if start > end:
#             print('不合适')
            continue
        if start < len_line_1 :
#             print('不合适')
            continue

        if end_index >= len(tokenized_text):
            continue
#         if start_index not in feature.token_to_orig_map:
#             continue
#         if end_index not in feature.token_to_orig_map:
#             continue
#         if not feature.token_is_max_context.get(start_index, False):
#             continue
#         if end_index < start_index:
#             continue
        length = end_index - start_index + 1
        if length > max_answer_length:
            continue
        else:
            if start_logits[start]<0:
                continue
            if start_logits[start]<0.1:
                continue
            if end_logits[end]<0:
                continue
            if end_logits[end]<0.1:
                continue
            text = tokenized_text[start:end]
            print('预测结果',text)
            key=lambda x: (x.start_logits[start] + x.start_logits[start]),
            
            print("start_logits",start_logits[start])
            print("end_logits",end_logits[end])
            

        
#         if start_index >= len(indexed_tokens):
#             continue
#         if end_index >= len(indexed_tokens):
#             continue
#         if start_index not in feature.token_to_orig_map:
#             continue
#         if end_index not in feature.token_to_orig_map:
#             continue
#         if not feature.token_is_max_context.get(start_index, False):
#             continue
#         if end_index < start_index:
#             continue
#         length = end_index - start_index + 1
#         if length > max_answer_length:
#             continue
#         prelim_predictions.append(
#             _PrelimPrediction(
#                 feature_index=feature_index,
#                 start_index=start_index,
#                 end_index=end_index,
#                 start_logit=result.start_logits[start_index],
#                 end_logit=result.end_logits[end_index]))

tokenized_text ['[CLS]', '明', '天', '天', '气', '怎', '么', '样', '[SEP]', '明', '天', '天', '气', '不', '错', '我', '们', '不', '如', '一', '起', '出', '去', '吧', '[SEP]']
预测结果 ['明', '天']
预测结果 ['不', '错', '我', '们', '不', '如', '一', '起']
start_logits 0.8450657725334167
end_logits 0.6883697509765625
预测结果 ['不', '错', '我', '们']
start_logits 0.8450657725334167
end_logits 0.6593695878982544
预测结果 ['不', '错', '我', '们', '不', '如', '一', '起', '出', '去']
start_logits 0.8450657725334167
end_logits 0.609614372253418
预测结果 []
start_logits 0.8450657725334167
end_logits 0.5905374884605408
预测结果 ['不', '错']
start_logits 0.8450657725334167
end_logits 0.5825162529945374
预测结果 ['不']
start_logits 0.8450657725334167
end_logits 0.5550487637519836
预测结果 ['不', '错', '我', '们', '不', '如', '一', '起', '出', '去', '吧']
start_logits 0.8450657725334167
end_logits 0.5493967533111572
预测结果 ['不', '错', '我', '们', '不', '如', '一', '起', '出']
start_logits 0.8450657725334167
end_logits 0.517236590385437
预测结果 ['不', '错', '我', '们', '不']
start_logits 0.8450657725334167

In [7]:
batch_start_logits[0, 0]

tensor(0.2047, device='cuda:0', grad_fn=<SelectBackward>)